# Benchmark Results Viewer

This notebook displays benchmark results in a nicely formatted table.

## Usage
1. Specify JSON file path OR paste the JSON output from the benchmark (summary) log
2. Run the cells to see the formatted output

In [1]:
import json
from pathlib import Path
from typing import Any

## Specify file or paste your benchmark JSON here

In [12]:
# Load from file
benchmark_file = Path("/Users/pokorny/JupyterLab/git/obi-output/circuit_extraction_benchmark_test4/initialize.circuit=N_10__top_nodes_dim6/neuron_set.sample_percentage=50.0__BENCHMARK__/benchmark_results.json")
with benchmark_file.open("r") as f:
    benchmark_data = json.load(f)

# OR paste the JSON from the benchmark summary log here
# benchmark_json = '{\"benchmarks\": {\"split_subcircuit\": {\"duration_s\": 0.43, \"mem_before_mb\": 375.49, \"mem_after_mb\": 385.45, \"mem_delta_mb\": 9.96, \"peak_mem_mb\": 385.45}}, \"benchmarked_time_s\": 0.43, \"total_execution_time_s\": 100.0, \"unbenchmarked_time_s\": 99.57, \"unbenchmarked_percentage\": 99.6}'
# benchmark_data = json.loads(benchmark_json)

## Display formatted results

In [13]:
def format_benchmark_results(benchmark_data: dict[str, Any]) -> None:
    """Display benchmark results in a nicely formatted table."""
    # Check if this is a summary (has 'benchmarks' key) or individual section
    if "benchmarks" in benchmark_data:
        # This is a summary
        benchmarks = benchmark_data["benchmarks"]
        benchmarked_time = benchmark_data.get("benchmarked_time_s")
        total_execution_time = benchmark_data.get("total_execution_time_s")
        unbenchmarked_time = benchmark_data.get("unbenchmarked_time_s")
        unbenchmarked_pct = benchmark_data.get("unbenchmarked_percentage")
        is_summary = True
    else:
        # This is an individual section benchmark
        benchmarks = benchmark_data
        benchmarked_time = sum(b["duration_s"] for b in benchmarks.values())
        total_execution_time = None
        unbenchmarked_time = None
        unbenchmarked_pct = None
        is_summary = False

    if not benchmarks:
        print("No benchmark data found.")
        return

    print("=" * 80)
    if is_summary:
        print("BENCHMARK SUMMARY")
    else:
        print("BENCHMARK SECTION")
    print("=" * 80)

    for name, data in benchmarks.items():
        duration = data['duration_s']
        # Calculate percentage if total execution time is available
        if total_execution_time and total_execution_time > 0:
            pct = (duration / total_execution_time) * 100
            pct_str = f" ({pct:.1f}%)"
        else:
            pct_str = ""
        
        print(f"\n{name}:")
        print(f"  Duration:      {duration:>10.2f} s{pct_str}")
        print(f"  Memory Before: {data['mem_before_mb']:>10.2f} MB")
        print(f"  Memory After:  {data['mem_after_mb']:>10.2f} MB")
        print(f"  Memory Delta:  {data['mem_delta_mb']:>10.2f} MB")
        print(f"  Peak Memory:   {data['peak_mem_mb']:>10.2f} MB")

    if is_summary:
        print(f"\n{'─' * 80}")
        if benchmarked_time is not None:
            print(f"Benchmarked Time: {benchmarked_time:.2f} s")
        
        if total_execution_time is not None:
            print(f"\nUnbenchmarked Time: {unbenchmarked_time:.2f} s ({unbenchmarked_pct:.1f}%)")
            print(f"Total Execution Time: {total_execution_time:.2f} s")
    print("=" * 80)

In [14]:
# Display
try:
    format_benchmark_results(benchmark_data)
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    print("Please check that you've pasted valid JSON.")

BENCHMARK SUMMARY

split_subcircuit:
  Duration:            0.11 s (9.9%)
  Memory Before:     357.19 MB
  Memory After:      363.91 MB
  Memory Delta:        6.72 MB
  Peak Memory:       363.91 MB

────────────────────────────────────────────────────────────────────────────────
Benchmarked Time: 0.11 s

Unbenchmarked Time: 1.00 s (90.1%)
Total Execution Time: 1.11 s
